In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input

def load_srcnn_model():
    input_shape = (None, None, 1)
    inputs = Input(shape=input_shape)
    conv1 = tf.keras.layers.Conv2D(128, (9, 9), activation='relu', padding='same')(inputs)
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    outputs = tf.keras.layers.Conv2D(1, (5, 5), padding='same')(conv2)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.load_weights(r'srcnn_weights.h5')  # 사전 학습된 가중치 파일의 경로
    return model



def enhance_image(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image)
    y = y.astype(np.float32) / 255.0
    y = np.expand_dims(np.expand_dims(y, axis=0), axis=-1)
    y_enhanced = model.predict(y)
    y_enhanced = y_enhanced[0, :, :, 0]
    y_enhanced = (y_enhanced * 255.0).clip(0, 255).astype(np.uint8)
    enhanced_image = cv2.merge([y_enhanced, cr, cb])
    enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_YCrCb2BGR)
    return enhanced_image

def enhance_images_in_directory(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    srcnn_model = load_srcnn_model()
    print('model load completed')
    
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(input_folder, image_file)
        print(f"Processing {image_path}")
        image = cv2.imread(image_path)
        
        if image is not None:
            enhanced_image = enhance_image(image, srcnn_model)
            output_path = os.path.join(output_folder, f"enhanced_{idx:04d}.jpg")
            cv2.imwrite(output_path, enhanced_image)
    
    print(f"Enhanced images saved to {output_folder}")

# 실행 예제 (현재 폴더 기준으로 input_images / output_images 폴더 사용)
input_folder = r'input_images'
output_folder = r'output_images'
enhance_images_in_directory(input_folder, output_folder)

model load completed
Processing input_images\input_images.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 759s 759s/step
Enhanced images saved to output_images


NameError: name 'model' is not defined

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
import gc

def load_srcnn_model():
    input_shape = (None, None, 1)
    inputs = Input(shape=input_shape)
    conv1 = tf.keras.layers.Conv2D(128, (9, 9), activation='relu', padding='same')(inputs)
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    outputs = tf.keras.layers.Conv2D(1, (5, 5), padding='same')(conv2)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.load_weights(r'srcnn_weights.h5')  # 사전 학습된 가중치 파일의 경로
    return model

def enhance_image(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image)
    y = y.astype(np.float32) / 255.0
    y = np.expand_dims(np.expand_dims(y, axis=0), axis=-1)
    y_enhanced = model.predict(y)
    y_enhanced = y_enhanced[0, :, :, 0]
    y_enhanced = (y_enhanced * 255.0).clip(0, 255).astype(np.uint8)
    enhanced_image = cv2.merge([y_enhanced, cr, cb])
    enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_YCrCb2BGR)
    return enhanced_image

def enhance_images_in_directory(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    srcnn_model = load_srcnn_model()

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.JPG'))]

    for idx, image_file in enumerate(image_files):
        print(f"ymshin input_folder {input_folder}")
        print(f"ymshin for {idx} {image_file}")
        image_path = os.path.join(input_folder, image_file)
        print(f"ymshin image_path = {image_path}")
        if os.path.exists(image_path):
            print("Image path exists. 이미지 있음")
        else:
            print("Image path does not exist. Check the path.")
            continue

        image = cv2.imread(image_path)

        if image is not None:
            print("img is not none 있어서 go")
            enhanced_image = enhance_image(image, srcnn_model)
            output_path = os.path.join(output_folder, f"enhanced_{idx:04d}.jpg")
            cv2.imwrite(output_path, enhanced_image)

            # 메모리 해제
            del image
            del enhanced_image
            gc.collect()

        # TensorFlow 세션을 주기적으로 클리어
        tf.keras.backend.clear_session()

    print(f"Enhanced images saved to {output_folder}")

# 사용 예제
input_folder = r'input_images'
output_folder = r'output_images'

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # 원하는 메모리 크기로 설정
            )
    except RuntimeError as e:
        print(e)

enhance_images_in_directory(input_folder, output_folder)

# GPU 메모리 클리어 (전체 작업이 끝난 후)
from numba import cuda
cuda.select_device(0)
cuda.close()

# TensorFlow 세션 종료 (전체 작업이 끝난 후)
tf.keras.backend.clear_session()

ymshin input_folder input_images
ymshin for 0 input_images.png
ymshin image_path = input_images\input_images.png
Image path exists. 이미지 있음
img is not none 있어서 go


In [4]:
!pip install numba

  Using cached numba-0.61.2-cp310-cp310-win_amd64.whl.metadata (2.9 kB)
  Using cached llvmlite-0.44.0-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------- ----------------- 1.6/2.8 MB 8.4 MB/s eta 0:00:01
   ------------------------------------- -- 2.6/2.8 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   - -------------------------------------- 1.0/30.3 MB 7.2 MB/s eta 0:00:05
   --- ------------------------------------ 2.9/30.3 MB 7.6 MB/s eta 0:00:04
   ------ --------------------------------- 5.0/30.3 MB 8.6 MB/s eta 0:00:03
   --------- ------------------------------ 7.3/30.3 MB 9.4 MB/s eta 0:00:03
   ------------ --------------------------- 9.4/30.3 MB 9.3 MB/s eta 0:00:03
   --------------- ------------------------ 11.5/30.3 MB 9.5 MB/s eta 0:00:02
   --------------- ----

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input

def load_srcnn_model():
    inputs = Input(shape=(None, None, 1))
    x = tf.keras.layers.Conv2D(128, (9, 9), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    outputs = tf.keras.layers.Conv2D(1, (5, 5), padding='same')(x)
    model = tf.keras.models.Model(inputs, outputs)
    model.load_weights('srcnn_weights.h5')  # 같은 폴더에 있어야 함
    return model

def enhance_image(image, model):
    image_ycc = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(image_ycc)
    y = y.astype(np.float32) / 255.0
    y = np.expand_dims(np.expand_dims(y, axis=0), axis=-1)
    y_enhanced = model.predict(y)[0, :, :, 0]
    y_enhanced = (y_enhanced * 255).clip(0, 255).astype(np.uint8)
    result = cv2.merge([y_enhanced, cr, cb])
    result = cv2.cvtColor(result, cv2.COLOR_YCrCb2BGR)
    return result

def enhance_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    model = load_srcnn_model()

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            if image is not None:
                enhanced = enhance_image(image, model)
                save_path = os.path.join(output_folder, f"enhanced_{filename}")
                cv2.imwrite(save_path, enhanced)

    print("모든 이미지 처리 완료!")

# 경로 지정
input_folder = 'input_images'
output_folder = 'output_images'

enhance_images(input_folder, output_folder)
